In [14]:
import transformers
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval, BertForSequenceClassification)
import torch
import numpy as np

In [180]:
#ckpt_dir = "/home/transformers-public/output_dirs/sarah-test6/checkpoint-6000" # Good!
# [417, 85]
# description [421, 81]

ckpt_dir = "/home/transformers-public/output_dirs/sarah-test7/checkpoint-4000" # Good!
# descruotiib [447, 55]
# title only [448, 54]

# ckpt_dir = "../output_dirs/classification_test/checkpoint-4500"

In [181]:
config = AutoConfig.from_pretrained(ckpt_dir)
tokenizer = BertTokenizer.from_pretrained(
    ckpt_dir,
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    ckpt_dir,
    from_tf=False,
    config=config,
)

In [182]:
use_gpu = True
if use_gpu:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda


In [183]:
model = model.to(device)

In [149]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    [("test input", None)],
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [124]:
model_inp = dict(test_input)
model_inp['labels'] = [0]
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [125]:
len(model_out)

3

In [35]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([1, 2]), torch.Size([1, 768])]


In [36]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits)

In [37]:
print(predicted_logits)

[[ 0.43291587 -0.18428318]]


In [26]:
print(predicted_class)

1


# Run on Doordash Data

In [73]:
import csv

dataset = []
with open("doordash_healthy.csv", newline='',encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dataset.append({
            'title': row['title'],
            'description': row['description'],
        })
        


In [188]:
labels = ['healthy', 'unhealthy']

counts = [0, 0]
for p in dataset:
    text = p['title'].lower() #+ ' ' + p['description'].lower()
    test_input = tokenizer.batch_encode_plus(
        [(text, None)],
        max_length=max_seq_length,
        pad_to_max_length=True,
    )
    
    model_inp = dict(test_input)
    model_inp['labels'] = [0]
    model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
    model_out = model(**model_inp)

    predicted_logits = model_out[1].detach().cpu().numpy()
    predicted_class = np.argmax(predicted_logits)
    
    #if labels[predicted_class] == 'unhealthy':
        #print(text)
    print(labels[predicted_class])
    #print(labels[predicted_class], predicted_logits)
    counts[predicted_class] += 1
    #print()


healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
unhealthy
healthy
healthy
healthy
unhealthy
healthy
healthy
unhealthy
unhealthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
unhealthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
unhealthy
unhealthy
healthy
healthy
healthy
healthy
unhealthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
unhealthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
unhealthy
unhealthy
unhealthy
healthy
healthy
unhealthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
health

In [185]:
counts

# descruotiib [447, 55]
# title only [448, 54]

[447, 55]

# Sanity Check 

In [186]:
labels = ['healthy', 'unhealthy']

samples = [
    "chocolate ice cream",
    "big mac", 
    "coca cola", 
    "fried oreos", 
    "salad", 
    "ramen",
    "beer", 
    "gyro sandwich",
    "avoca"
]

for text in samples:

    test_input = tokenizer.batch_encode_plus(
        [(text, None)],
        max_length=max_seq_length,
        pad_to_max_length=True,
    )
    
    model_inp = dict(test_input)
    model_inp['labels'] = [0]
    model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
    model_out = model(**model_inp)

    predicted_logits = model_out[1].detach().cpu().numpy()
    predicted_class = np.argmax(predicted_logits)
    
    print(text)
    print(labels[predicted_class], predicted_logits)
    print()

chocolate ice cream
unhealthy [[-0.21458574  0.15276423]]

big mac
unhealthy [[-3.6606882  3.216692 ]]

coca cola
unhealthy [[-0.39577508  0.39863577]]

fried oreos
unhealthy [[-0.22029486  0.32009664]]

salad
healthy [[ 0.56455594 -0.41481605]]

ramen
healthy [[ 0.63726187 -0.4711248 ]]

beer
unhealthy [[-0.00074588  0.12836695]]

gyro sandwich
healthy [[ 0.48251906 -0.34447554]]

